In [0]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd


mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

df_sales = pd.read_csv("/content/drive/My Drive/DataAnalytics/sales_train_validation.csv")
df_prices= pd.read_csv("/content/drive/My Drive/DataAnalytics/sell_prices.csv")
df_cal= pd.read_csv("/content/drive/My Drive/DataAnalytics/calendar.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Saving all the IDs of all the products and converting the index into a column
df_sales = df_sales.reset_index()
ids = pd.DataFrame(df_sales['index'])
ids['id'] = df_sales['id']
ids['id'] = df_sales['id'].map(lambda x: x.rstrip('_validation'))
ids.head()

,index,id
0,0,HOBBIES_1_001_CA_1
1,1,HOBBIES_1_002_CA_1
2,2,HOBBIES_1_003_CA_1
3,3,HOBBIES_1_004_CA_1
4,4,HOBBIES_1_005_CA_1


In [0]:
# We transponse the table and only get all the rows that correspond to the days and get the index to a column to use it to merge it wiht 
# the calendar so that we could get the week that each day belong to
df_sales = df_sales.T
sales_amount = df_sales[7:]
sales_amount = sales_amount.reset_index()
sales_amount.head()

,index,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,30450,30451,30452,30453,30454,30455,30456,30457,30458,30459,30460,30461,30462,30463,30464,30465,30466,30467,30468,30469,30470,30471,30472,30473,30474,30475,30476,30477,30478,30479,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
0,d_1,0,0,0,0,0,0,0,12,2,0,0,0,0,0,4,5,0,0,0,0,0,2,2,0,0,0,0,0,2,0,0,9,0,0,0,2,0,0,0,...,0,0,0,0,16,2,1,0,0,0,0,0,36,0,0,0,14,0,0,7,4,0,0,0,0,0,0,0,0,0,0,14,1,0,4,0,0,0,0,0
1,d_2,0,0,0,0,0,0,0,15,0,0,0,2,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,12,6,0,0,0,0,0,0,39,0,0,0,7,0,0,6,13,0,0,0,4,0,0,0,0,1,0,11,1,0,4,0,0,6,0,0
2,d_3,0,0,0,0,0,0,0,0,7,1,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0,0,0,0,3,0,0,5,0,0,0,0,0,1,0,...,0,0,0,0,11,1,1,0,0,0,0,0,34,0,0,0,6,0,0,9,5,0,0,0,0,0,0,0,0,0,0,5,1,0,2,2,0,0,0,0
3,d_4,0,0,0,0,0,0,0,0,3,0,0,0,0,0,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,16,0,0,0,0,0,1,0,...,0,0,0,0,11,3,1,0,0,0,0,0,42,0,0,0,9,0,0,1,1,0,0,0,1,0,0,0,0,1,0,6,1,0,5,2,0,2,0,0
4,d_5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,8,0,0,0,0,0,0,0,...,0,0,0,0,6,5,0,0,0,0,0,0,29,0,0,0,0,0,0,6,5,0,0,0,0,0,0,0,0,0,0,5,1,0,2,0,0,2,0,0


In [0]:
# We merge the sales table with the calendar and only keep the week column from the calendar's table
sales_amount.rename(columns={'index' : 'd' }, inplace = True)
sales_amount = pd.merge(sales_amount,df_cal, how='outer')
sales_amount.drop(columns={'wday','date','weekday','month','year','event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI'}, inplace=True)
sales_amount


,d,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,30451,30452,30453,30454,30455,30456,30457,30458,30459,30460,30461,30462,30463,30464,30465,30466,30467,30468,30469,30470,30471,30472,30473,30474,30475,30476,30477,30478,30479,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489,wm_yr_wk
0,d_1,0,0,0,0,0,0,0,12,2,0,0,0,0,0,4,5,0,0,0,0,0,2,2,0,0,0,0,0,2,0,0,9,0,0,0,2,0,0,0,...,0,0,0,16,2,1,0,0,0,0,0,36,0,0,0,14,0,0,7,4,0,0,0,0,0,0,0,0,0,0,14,1,0,4,0,0,0,0,0,11101
1,d_2,0,0,0,0,0,0,0,15,0,0,0,2,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,12,6,0,0,0,0,0,0,39,0,0,0,7,0,0,6,13,0,0,0,4,0,0,0,0,1,0,11,1,0,4,0,0,6,0,0,11101
2,d_3,0,0,0,0,0,0,0,0,7,1,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0,0,0,0,3,0,0,5,0,0,0,0,0,1,0,...,0,0,0,11,1,1,0,0,0,0,0,34,0,0,0,6,0,0,9,5,0,0,0,0,0,0,0,0,0,0,5,1,0,2,2,0,0,0,0,11101
3,d_4,0,0,0,0,0,0,0,0,3,0,0,0,0,0,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,16,0,0,0,0,0,1,0,...,0,0,0,11,3,1,0,0,0,0,0,42,0,0,0,9,0,0,1,1,0,0,0,1,0,0,0,0,1,0,6,1,0,5,2,0,2,0,0,11101
4,d_5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,8,0,0,0,0,0,0,0,...,0,0,0,6,5,0,0,0,0,0,0,29,0,0,0,0,0,0,6,5,0,0,0,0,0,0,0,0,0,0,5,1,0,2,0,0,2,0,0,11101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,d_1965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11620
1965,d_1966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11620
1966,d_1967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11620
1967,d_1968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11621


In [0]:
df_sales.T

,index,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,0,3,1,4,1,0,0,3,4,4,0,0,1,0,1,1,7,7,3,6,3,3,7,12,4,2,7,5,12,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,1,0,0,1
30486,30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,5,0,1,1,3,1,1,0,4,2,0,1,2,1,1,0,0,0,0,3,1,1,1,2,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0
30487,30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,2,4,1,8,5,2,7,5,3,5,20,8,10,3,3,4,7,2,3,5,6,3,4,1,2,5,1,2,2,...,1,1,2,0,0,2,0,0,0,2,0,1,1,0,0,0,0,1,1,2,1,4,0,0,2,2,4,0,0,0,2,1,0,2,0,1,0,0,1,0
30488,30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,0,2,1,1,2,4,3,0,0,0,0,0,1,1,2,1,1,1,1,2,0,1,0,3,0,0,1,0,0,1,0,3,1,3


In [0]:
# We clean the result we got after meging the two tables by droping all the null values, to do that we first had to fill in all na whith the fillna function
# then we just looked for all the rows containing the value we used to fill in the NAs and drop them.
sales_amount = sales_amount.T
sales_amount = sales_amount.fillna(-999) 
sales_amount.drop(columns=sales_amount.columns[(sales_amount == -999).any()], inplace=True)
sales_amount

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1873,1874,1875,1876,1877,1878,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1899,1900,1901,1902,1903,1904,1905,1906,1907,1908,1909,1910,1911,1912
d,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,...,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30486,0,0,0,0,0,5,0,1,1,3,1,1,0,4,2,0,1,2,1,1,0,0,0,0,3,1,1,1,2,0,1,0,1,1,1,3,3,4,4,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0
30487,0,6,0,2,2,4,1,8,5,2,7,5,3,5,20,8,10,3,3,4,7,2,3,5,6,3,4,1,2,5,1,2,2,2,1,9,8,1,6,3,...,1,1,2,0,0,2,0,0,0,2,0,1,1,0,0,0,0,1,1,2,1,4,0,0,2,2,4,0,0,0,2,1,0,2,0,1,0,0,1,0
30488,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,0,2,1,1,2,4,3,0,0,0,0,0,1,1,2,1,1,1,1,2,0,1,0,3,0,0,1,0,0,1,0,3,1,3
30489,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,3,0,1,4,2,0,5,3,2,0,5,7,3,1,4,3,3,1,2,3,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Here we rename the columns with the values of the d row and drop that row since we don't need it any more. Also we reset the index and we used this
# column to merge it with the id table that we created above.
sales_amount = sales_amount.rename(columns=sales_amount.iloc[0]).drop(sales_amount.index[0])
sales_amount = sales_amount.reset_index()
sales_amount

,index,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,...,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30486,30486,0,0,0,0,0,5,0,1,1,3,1,1,0,4,2,0,1,2,1,1,0,0,0,0,3,1,1,1,2,0,1,0,1,1,1,3,3,4,4,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0
30487,30487,0,6,0,2,2,4,1,8,5,2,7,5,3,5,20,8,10,3,3,4,7,2,3,5,6,3,4,1,2,5,1,2,2,2,1,9,8,1,6,...,1,1,2,0,0,2,0,0,0,2,0,1,1,0,0,0,0,1,1,2,1,4,0,0,2,2,4,0,0,0,2,1,0,2,0,1,0,0,1,0
30488,30488,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,0,2,1,1,2,4,3,0,0,0,0,0,1,1,2,1,1,1,1,2,0,1,0,3,0,0,1,0,0,1,0,3,1,3
30489,30489,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,3,0,1,4,2,0,5,3,2,0,5,7,3,1,4,3,3,1,2,3,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Here we merged the sales tables with the ids talbe just to add the item id to all our products
sales_amount = pd.merge(sales_amount,ids, how='outer')
sales_amount

,index,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,...,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,id
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1,HOBBIES_1_001_CA_1
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,HOBBIES_1_002_CA_1
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1,HOBBIES_1_003_CA_1
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,...,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2,HOBBIES_1_004_CA_1
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4,HOBBIES_1_005_CA_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30486,30486,0,0,0,0,0,5,0,1,1,3,1,1,0,4,2,0,1,2,1,1,0,0,0,0,3,1,1,1,2,0,1,0,1,1,1,3,3,4,4,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,FOODS_3_824_WI_3
30487,30487,0,6,0,2,2,4,1,8,5,2,7,5,3,5,20,8,10,3,3,4,7,2,3,5,6,3,4,1,2,5,1,2,2,2,1,9,8,1,6,...,1,2,0,0,2,0,0,0,2,0,1,1,0,0,0,0,1,1,2,1,4,0,0,2,2,4,0,0,0,2,1,0,2,0,1,0,0,1,0,FOODS_3_825_WI_3
30488,30488,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,0,2,1,1,2,4,3,0,0,0,0,0,1,1,2,1,1,1,1,2,0,1,0,3,0,0,1,0,0,1,0,3,1,3,FOODS_3_826_WI_3
30489,30489,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,0,1,4,2,0,5,3,2,0,5,7,3,1,4,3,3,1,2,3,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,FOODS_3_827_WI_3


In [0]:
# we create a new dataframe with only wm_yr_wk and index columns to merge it with the prices table
sale3 = sales_amount.T
sale3 = sale3.reset_index()
day_week = pd.DataFrame(sale3[30490])
day_week['day'] = sale3['index']
day_week = day_week.rename(columns=day_week.iloc[0]).drop(day_week.index[0])
day_week = day_week.drop(day_week.index[1913])
day_week.drop(day_week.loc[day_week['index']=='id'].index, inplace=True)
day_week

,wm_yr_wk,index
1,11101,d_1
2,11101,d_2
3,11101,d_3
4,11101,d_4
5,11101,d_5
...,...,...
1909,11612,d_1909
1910,11612,d_1910
1911,11612,d_1911
1912,11613,d_1912


In [0]:
# We load the sales_train_validation dataset again
df_sales = pd.read_csv("/content/drive/My Drive/DataAnalytics/sales_train_validation.csv")
df_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4


In [0]:
# We drop the columns that we don't need and transpose the matrix 
df1 = df_sales.drop(['item_id','dept_id','cat_id','store_id','state_id'],axis=1)
df1['id'] = df_sales['id'].map(lambda x: x.rstrip('_validation'))

df1 = df1.T
df1 = df1.rename(columns=df1.iloc[0]).drop(df1.index[0])
df1.index.names = [None]
df1.index = np.arange(1,len(df1)+1)
df1.reset_index(inplace=True)
df1.rename(columns={"index": "Day"}, inplace=True)
print(df1.shape)
df1.head()

(1913, 30491)


,Day,HOBBIES_1_001_CA_1,HOBBIES_1_002_CA_1,HOBBIES_1_003_CA_1,HOBBIES_1_004_CA_1,HOBBIES_1_005_CA_1,HOBBIES_1_006_CA_1,HOBBIES_1_007_CA_1,HOBBIES_1_008_CA_1,HOBBIES_1_009_CA_1,HOBBIES_1_010_CA_1,HOBBIES_1_011_CA_1,HOBBIES_1_012_CA_1,HOBBIES_1_013_CA_1,HOBBIES_1_014_CA_1,HOBBIES_1_015_CA_1,HOBBIES_1_016_CA_1,HOBBIES_1_017_CA_1,HOBBIES_1_018_CA_1,HOBBIES_1_019_CA_1,HOBBIES_1_020_CA_1,HOBBIES_1_021_CA_1,HOBBIES_1_022_CA_1,HOBBIES_1_023_CA_1,HOBBIES_1_024_CA_1,HOBBIES_1_025_CA_1,HOBBIES_1_026_CA_1,HOBBIES_1_027_CA_1,HOBBIES_1_028_CA_1,HOBBIES_1_029_CA_1,HOBBIES_1_030_CA_1,HOBBIES_1_031_CA_1,HOBBIES_1_032_CA_1,HOBBIES_1_033_CA_1,HOBBIES_1_034_CA_1,HOBBIES_1_035_CA_1,HOBBIES_1_036_CA_1,HOBBIES_1_037_CA_1,HOBBIES_1_038_CA_1,HOBBIES_1_039_CA_1,...,FOODS_3_788_WI_3,FOODS_3_789_WI_3,FOODS_3_790_WI_3,FOODS_3_791_WI_3,FOODS_3_792_WI_3,FOODS_3_793_WI_3,FOODS_3_794_WI_3,FOODS_3_795_WI_3,FOODS_3_796_WI_3,FOODS_3_797_WI_3,FOODS_3_798_WI_3,FOODS_3_799_WI_3,FOODS_3_800_WI_3,FOODS_3_801_WI_3,FOODS_3_802_WI_3,FOODS_3_803_WI_3,FOODS_3_804_WI_3,FOODS_3_805_WI_3,FOODS_3_806_WI_3,FOODS_3_807_WI_3,FOODS_3_808_WI_3,FOODS_3_809_WI_3,FOODS_3_810_WI_3,FOODS_3_811_WI_3,FOODS_3_812_WI_3,FOODS_3_813_WI_3,FOODS_3_814_WI_3,FOODS_3_815_WI_3,FOODS_3_816_WI_3,FOODS_3_817_WI_3,FOODS_3_818_WI_3,FOODS_3_819_WI_3,FOODS_3_820_WI_3,FOODS_3_821_WI_3,FOODS_3_822_WI_3,FOODS_3_823_WI_3,FOODS_3_824_WI_3,FOODS_3_825_WI_3,FOODS_3_826_WI_3,FOODS_3_827_WI_3
0,1,0,0,0,0,0,0,0,12,2,0,0,0,0,0,4,5,0,0,0,0,0,2,2,0,0,0,0,0,2,0,0,9,0,0,0,2,0,0,0,...,0,0,0,0,16,2,1,0,0,0,0,0,36,0,0,0,14,0,0,7,4,0,0,0,0,0,0,0,0,0,0,14,1,0,4,0,0,0,0,0
1,2,0,0,0,0,0,0,0,15,0,0,0,2,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,12,6,0,0,0,0,0,0,39,0,0,0,7,0,0,6,13,0,0,0,4,0,0,0,0,1,0,11,1,0,4,0,0,6,0,0
2,3,0,0,0,0,0,0,0,0,7,1,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0,0,0,0,3,0,0,5,0,0,0,0,0,1,0,...,0,0,0,0,11,1,1,0,0,0,0,0,34,0,0,0,6,0,0,9,5,0,0,0,0,0,0,0,0,0,0,5,1,0,2,2,0,0,0,0
3,4,0,0,0,0,0,0,0,0,3,0,0,0,0,0,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,16,0,0,0,0,0,1,0,...,0,0,0,0,11,3,1,0,0,0,0,0,42,0,0,0,9,0,0,1,1,0,0,0,1,0,0,0,0,1,0,6,1,0,5,2,0,2,0,0
4,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,8,0,0,0,0,0,0,0,...,0,0,0,0,6,5,0,0,0,0,0,0,29,0,0,0,0,0,0,6,5,0,0,0,0,0,0,0,0,0,0,5,1,0,2,0,0,2,0,0


In [0]:
# Load the sell_prices dataset and create a new column id using the item_id and the store_id, we just kept the id column, wm_yr_wk, sell_price and drop all 
# the others.
df_prices = pd.read_csv("/content/drive/My Drive/DataAnalytics/sell_prices.csv")
df_prices["id"] = df_prices['item_id'] + '_' + df_prices['store_id']
df_prices = df_prices.drop(['store_id','item_id'],axis=1)
df_prices = df_prices[['id','wm_yr_wk','sell_price']]
df_prices.head()

,id,wm_yr_wk,sell_price
0,HOBBIES_1_001_CA_1,11325,9.58
1,HOBBIES_1_001_CA_1,11326,9.58
2,HOBBIES_1_001_CA_1,11327,8.26
3,HOBBIES_1_001_CA_1,11328,8.26
4,HOBBIES_1_001_CA_1,11329,8.26


In [0]:
# We merged the prices and the weekday dataset so that we know the price of each item for each week
df_price_by_day = pd.merge(df_prices, day_week, on='wm_yr_wk', how='outer')
df_price_by_day = df_price_by_day.rename(columns={'index': 'Day'})
df_price_by_day.head()

,id,wm_yr_wk,sell_price,Day
0,HOBBIES_1_001_CA_1,11325,9.58,d_897
1,HOBBIES_1_001_CA_1,11325,9.58,d_898
2,HOBBIES_1_001_CA_1,11325,9.58,d_899
3,HOBBIES_1_001_CA_1,11325,9.58,d_900
4,HOBBIES_1_001_CA_1,11325,9.58,d_901


In [0]:
# Created a matrix with the prices of each item on each day
df_itemprice_by_day = df_price_by_day.pivot_table(index='id',columns='Day',values='sell_price')
df_itemprice_by_day.head()

Day,d_1,d_10,d_100,d_1000,d_1001,d_1002,d_1003,d_1004,d_1005,d_1006,d_1007,d_1008,d_1009,d_101,d_1010,d_1011,d_1012,d_1013,d_1014,d_1015,d_1016,d_1017,d_1018,d_1019,d_102,d_1020,d_1021,d_1022,d_1023,d_1024,d_1025,d_1026,d_1027,d_1028,d_1029,d_103,d_1030,d_1031,d_1032,d_1033,...,d_963,d_964,d_965,d_966,d_967,d_968,d_969,d_97,d_970,d_971,d_972,d_973,d_974,d_975,d_976,d_977,d_978,d_979,d_98,d_980,d_981,d_982,d_983,d_984,d_985,d_986,d_987,d_988,d_989,d_99,d_990,d_991,d_992,d_993,d_994,d_995,d_996,d_997,d_998,d_999
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1,2.0,2.0,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
FOODS_1_001_CA_2,2.0,2.0,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
FOODS_1_001_CA_3,2.0,2.0,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.00,2.00,2.00,2.0,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.0,2.00,2.00,2.00,2.00,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
FOODS_1_001_CA_4,2.0,2.0,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
FOODS_1_001_TX_1,2.0,2.0,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.0,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24


In [0]:
df_itemprice_by_day.shape

(30490, 1913)

In [0]:
new_sales_amount = sales_amount.drop(['index'], axis=1)
new_sales_amount

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,...,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,id
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1,HOBBIES_1_001_CA_1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,HOBBIES_1_002_CA_1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1,HOBBIES_1_003_CA_1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,...,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2,HOBBIES_1_004_CA_1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4,HOBBIES_1_005_CA_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30486,0,0,0,0,0,5,0,1,1,3,1,1,0,4,2,0,1,2,1,1,0,0,0,0,3,1,1,1,2,0,1,0,1,1,1,3,3,4,4,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,FOODS_3_824_WI_3
30487,0,6,0,2,2,4,1,8,5,2,7,5,3,5,20,8,10,3,3,4,7,2,3,5,6,3,4,1,2,5,1,2,2,2,1,9,8,1,6,3,...,1,2,0,0,2,0,0,0,2,0,1,1,0,0,0,0,1,1,2,1,4,0,0,2,2,4,0,0,0,2,1,0,2,0,1,0,0,1,0,FOODS_3_825_WI_3
30488,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,0,2,1,1,2,4,3,0,0,0,0,0,1,1,2,1,1,1,1,2,0,1,0,3,0,0,1,0,0,1,0,3,1,3,FOODS_3_826_WI_3
30489,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,0,1,4,2,0,5,3,2,0,5,7,3,1,4,3,3,1,2,3,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,FOODS_3_827_WI_3


In [0]:
new_sales_amount = new_sales_amount[:-1]
new_sales_amount

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,...,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,id
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1,HOBBIES_1_001_CA_1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,HOBBIES_1_002_CA_1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1,HOBBIES_1_003_CA_1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,...,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2,HOBBIES_1_004_CA_1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4,HOBBIES_1_005_CA_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,2,2,0,3,1,4,1,0,0,3,4,4,0,0,1,0,1,1,7,7,3,6,3,3,7,12,4,2,7,5,12,5,3,3,7,6,4,6,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,1,0,0,1,FOODS_3_823_WI_3
30486,0,0,0,0,0,5,0,1,1,3,1,1,0,4,2,0,1,2,1,1,0,0,0,0,3,1,1,1,2,0,1,0,1,1,1,3,3,4,4,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,FOODS_3_824_WI_3
30487,0,6,0,2,2,4,1,8,5,2,7,5,3,5,20,8,10,3,3,4,7,2,3,5,6,3,4,1,2,5,1,2,2,2,1,9,8,1,6,3,...,1,2,0,0,2,0,0,0,2,0,1,1,0,0,0,0,1,1,2,1,4,0,0,2,2,4,0,0,0,2,1,0,2,0,1,0,0,1,0,FOODS_3_825_WI_3
30488,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,0,2,1,1,2,4,3,0,0,0,0,0,1,1,2,1,1,1,1,2,0,1,0,3,0,0,1,0,0,1,0,3,1,3,FOODS_3_826_WI_3


In [0]:
# Set the id column as the index of the table
new_sales_amount = new_sales_amount.set_index('id')
new_sales_amount

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
HOBBIES_1_002_CA_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
HOBBIES_1_003_CA_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
HOBBIES_1_004_CA_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,...,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
HOBBIES_1_005_CA_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOODS_3_823_WI_3,0,0,2,2,0,3,1,4,1,0,0,3,4,4,0,0,1,0,1,1,7,7,3,6,3,3,7,12,4,2,7,5,12,5,3,3,7,6,4,6,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,1,0,0,1
FOODS_3_824_WI_3,0,0,0,0,0,5,0,1,1,3,1,1,0,4,2,0,1,2,1,1,0,0,0,0,3,1,1,1,2,0,1,0,1,1,1,3,3,4,4,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0
FOODS_3_825_WI_3,0,6,0,2,2,4,1,8,5,2,7,5,3,5,20,8,10,3,3,4,7,2,3,5,6,3,4,1,2,5,1,2,2,2,1,9,8,1,6,3,...,1,1,2,0,0,2,0,0,0,2,0,1,1,0,0,0,0,1,1,2,1,4,0,0,2,2,4,0,0,0,2,1,0,2,0,1,0,0,1,0


In [0]:
# Sort the indexes in descending mode in the itemprice_by_day table
df_itemprice_by_day.sort_index(inplace=True)
df_itemprice_by_day

Day,d_1,d_10,d_100,d_1000,d_1001,d_1002,d_1003,d_1004,d_1005,d_1006,d_1007,d_1008,d_1009,d_101,d_1010,d_1011,d_1012,d_1013,d_1014,d_1015,d_1016,d_1017,d_1018,d_1019,d_102,d_1020,d_1021,d_1022,d_1023,d_1024,d_1025,d_1026,d_1027,d_1028,d_1029,d_103,d_1030,d_1031,d_1032,d_1033,...,d_963,d_964,d_965,d_966,d_967,d_968,d_969,d_97,d_970,d_971,d_972,d_973,d_974,d_975,d_976,d_977,d_978,d_979,d_98,d_980,d_981,d_982,d_983,d_984,d_985,d_986,d_987,d_988,d_989,d_99,d_990,d_991,d_992,d_993,d_994,d_995,d_996,d_997,d_998,d_999
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1,2.00,2.00,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
FOODS_1_001_CA_2,2.00,2.00,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
FOODS_1_001_CA_3,2.00,2.00,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
FOODS_1_001_CA_4,2.00,2.00,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
FOODS_1_001_TX_1,2.00,2.00,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_516_TX_2,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,...,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94
HOUSEHOLD_2_516_TX_3,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,...,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94
HOUSEHOLD_2_516_WI_1,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5

In [0]:
# Sort the indexes in descending mode in the new_sales_amount table so that they match the same order of the previous table
new_sales_amount.sort_index(inplace=True)
new_sales_amount

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1,3,0,0,1,4,2,0,2,0,0,0,0,3,1,3,0,2,1,2,0,2,1,2,0,2,2,2,4,2,2,0,2,1,7,1,2,3,0,1,2,...,0,2,1,2,2,0,2,2,0,1,1,0,2,1,1,0,4,0,0,4,1,3,0,1,0,2,2,0,1,1,0,2,0,4,1,1,0,1,1,0
FOODS_1_001_CA_2,2,0,0,0,1,0,8,0,4,1,2,1,1,3,4,1,2,2,2,0,1,1,1,1,2,3,16,0,0,0,0,0,1,1,1,1,3,1,0,0,...,1,0,0,0,2,1,0,0,0,0,2,1,0,0,0,0,0,0,0,1,1,2,0,0,0,0,0,0,2,0,1,0,14,0,1,1,4,0,0,4
FOODS_1_001_CA_3,1,2,1,1,1,2,0,1,1,1,0,0,3,3,2,7,1,2,4,0,1,1,3,2,0,0,0,0,6,6,1,1,0,0,4,3,6,5,1,0,...,0,0,1,2,4,2,0,1,1,1,12,0,0,0,1,0,0,1,0,0,0,1,0,0,4,2,1,0,0,0,0,0,13,0,0,0,0,0,1,0
FOODS_1_001_CA_4,0,1,1,1,1,1,0,0,0,0,0,0,1,2,1,0,1,0,0,1,0,2,0,1,0,1,1,0,1,0,0,1,1,1,0,0,1,2,2,2,...,0,0,1,1,0,1,0,1,0,0,1,0,2,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,1,1
FOODS_1_001_TX_1,0,1,1,0,0,0,0,0,2,1,0,0,0,1,0,0,0,0,0,0,0,0,1,2,0,0,2,0,0,0,0,3,1,0,0,0,0,0,0,0,...,1,1,0,1,2,1,0,1,0,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_516_TX_2,0,1,0,0,0,0,1,1,0,0,0,1,1,1,1,3,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,1,0,0,1,1,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,0
HOUSEHOLD_2_516_TX_3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
HOUSEHOLD_2_516_WI_1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [0]:
df_itemprice_by_day.sort_index(axis=1, inplace=True)
df_itemprice_by_day

Day,d_1,d_10,d_100,d_1000,d_1001,d_1002,d_1003,d_1004,d_1005,d_1006,d_1007,d_1008,d_1009,d_101,d_1010,d_1011,d_1012,d_1013,d_1014,d_1015,d_1016,d_1017,d_1018,d_1019,d_102,d_1020,d_1021,d_1022,d_1023,d_1024,d_1025,d_1026,d_1027,d_1028,d_1029,d_103,d_1030,d_1031,d_1032,d_1033,...,d_963,d_964,d_965,d_966,d_967,d_968,d_969,d_97,d_970,d_971,d_972,d_973,d_974,d_975,d_976,d_977,d_978,d_979,d_98,d_980,d_981,d_982,d_983,d_984,d_985,d_986,d_987,d_988,d_989,d_99,d_990,d_991,d_992,d_993,d_994,d_995,d_996,d_997,d_998,d_999
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1,2.00,2.00,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
FOODS_1_001_CA_2,2.00,2.00,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
FOODS_1_001_CA_3,2.00,2.00,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
FOODS_1_001_CA_4,2.00,2.00,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
FOODS_1_001_TX_1,2.00,2.00,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.00,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_516_TX_2,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,...,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94
HOUSEHOLD_2_516_TX_3,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,...,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94
HOUSEHOLD_2_516_WI_1,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5

In [0]:
new_sales_amount.sort_index(axis=1, inplace=True)
new_sales_amount

,d_1,d_10,d_100,d_1000,d_1001,d_1002,d_1003,d_1004,d_1005,d_1006,d_1007,d_1008,d_1009,d_101,d_1010,d_1011,d_1012,d_1013,d_1014,d_1015,d_1016,d_1017,d_1018,d_1019,d_102,d_1020,d_1021,d_1022,d_1023,d_1024,d_1025,d_1026,d_1027,d_1028,d_1029,d_103,d_1030,d_1031,d_1032,d_1033,...,d_963,d_964,d_965,d_966,d_967,d_968,d_969,d_97,d_970,d_971,d_972,d_973,d_974,d_975,d_976,d_977,d_978,d_979,d_98,d_980,d_981,d_982,d_983,d_984,d_985,d_986,d_987,d_988,d_989,d_99,d_990,d_991,d_992,d_993,d_994,d_995,d_996,d_997,d_998,d_999
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1,3,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,1,1,0,0,3,1,1,0,0,0,0,0,0,0,9,1,0,0,0,0,0,2,0,4,2,0,0,0,1,0,2,0,0,0,1,0
FOODS_1_001_CA_2,2,1,5,0,6,1,4,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,3,4,1,5,0,0,0,2,0,0,5,2,2,0,0,6,0,1,0,1,0,1,0,2,0,3,1,2,0,3,2,1,1,1,0
FOODS_1_001_CA_3,1,1,1,0,1,3,1,0,1,0,1,3,2,5,3,0,0,0,0,0,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0,4,0,0,0,0,...,0,0,2,0,0,0,0,4,2,0,0,0,0,1,0,0,0,0,0,0,6,4,1,0,0,1,0,0,0,0,0,0,1,0,7,2,0,0,0,0
FOODS_1_001_CA_4,0,0,0,0,0,0,0,0,0,0,0,3,0,2,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,0,0,0,1,0,1,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,1,1,0,5,0,0,0,0,0,0,0,0,0,1
FOODS_1_001_TX_1,0,1,1,0,1,1,4,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,1,0,0,1,0,1,1,0,0,0,1,1,1,3,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_516_TX_2,0,0,0,0,0,2,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1
HOUSEHOLD_2_516_TX_3,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
HOUSEHOLD_2_516_WI_1,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,...,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Multiplied the df_itemprice_by_day and the new_sales_amount to get the total revenue each item got on each day
df_revenue = df_itemprice_by_day * new_sales_amount
df_revenue

Day,d_1,d_10,d_100,d_1000,d_1001,d_1002,d_1003,d_1004,d_1005,d_1006,d_1007,d_1008,d_1009,d_101,d_1010,d_1011,d_1012,d_1013,d_1014,d_1015,d_1016,d_1017,d_1018,d_1019,d_102,d_1020,d_1021,d_1022,d_1023,d_1024,d_1025,d_1026,d_1027,d_1028,d_1029,d_103,d_1030,d_1031,d_1032,d_1033,...,d_963,d_964,d_965,d_966,d_967,d_968,d_969,d_97,d_970,d_971,d_972,d_973,d_974,d_975,d_976,d_977,d_978,d_979,d_98,d_980,d_981,d_982,d_983,d_984,d_985,d_986,d_987,d_988,d_989,d_99,d_990,d_991,d_992,d_993,d_994,d_995,d_996,d_997,d_998,d_999
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1,6,0,0,2.24,0,2.24,0,0,0,0,2.24,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,...,0,0,0,0,2.24,2.24,0,0,6.72,2.24,2.24,0,0,0,0,0,0,0,18,2.24,0,0,0,0,0,4.48,0,8.96,4.48,0,0,0,2.24,0,4.48,0,0,0,2.24,0
FOODS_1_001_CA_2,4,2,10,0,13.44,2.24,8.96,2.24,2.24,0,2.24,0,0,0,0,2.24,0,0,0,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,2.24,2.24,0,0,6.72,8.96,2,11.2,0,0,0,4.48,0,0,11.2,4.48,4.48,0,0,13.44,0,2.24,0,2.24,0,2.24,0,4.48,0,6.72,2.24,4.48,0,6.72,4.48,2.24,2.24,2.24,0
FOODS_1_001_CA_3,2,2,2,0,2.24,6.72,2.24,0,2.24,0,2.24,6.72,4.48,10,6.72,0,0,0,0,0,6,0,0,0,0,0,0,0,6,0,0,0,0,0,0,8,0,0,0,0,...,0,0,4.48,0,0,0,0,8,4.48,0,0,0,0,2.24,0,0,0,0,0,0,13.44,8.96,2.24,0,0,2.24,0,0,0,0,0,0,2.24,0,15.68,4.48,0,0,0,0
FOODS_1_001_CA_4,0,0,0,0,0,0,0,0,0,0,0,6.72,0,4,4.48,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2.24,2.24,2.24,0,2.24,0,0,0,2.24,0,2.24,0,0,0,4.48,0,0,0,0,0,2.24,0,0,0,0,0,2.24,2.24,0,10,0,0,0,0,0,0,0,0,0,2.24
FOODS_1_001_TX_1,0,2,2,0,2.24,2.24,8.96,4.48,0,0,0,0,0,0,0,2.24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,...,0,0,0,2.24,0,2.24,0,2,0,0,2.24,0,2.24,2.24,0,0,0,2.24,2,2.24,6.72,0,0,0,0,0,0,0,0,2,2.24,2.24,0,2.24,0,0,0,2.24,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_516_TX_2,0,0,0,0,0,11.88,5.94,0,0,0,0,0,0,5.94,0,5.94,0,0,0,0,0,0,0,0,0,0,0,0,5.94,5.94,0,0,0,0,0,5.94,0,0,0,0,...,0,0,0,0,0,0,0,0,5.94,0,0,0,5.94,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.94,0,0,0,5.94,0,0,0,0,0,5.94,0,5.94
HOUSEHOLD_2_516_TX_3,11.88,0,0,0,0,5.94,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.94,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.94,0,0,5.94,0,0,0,0,0,5.94,0,0,0,0,0,0,0
HOUSEHOLD_2_516_WI_1,0,0,0,0,0,0,0,0,0,0,11.88,0,0,0,0,5.94,0,0,0,0,0,0,0,0,0,0,0,0,5.94,5.94,5.94,0,0,0,0,0,0,0,0,0,...,5.94,5.94,0,5.94,0,0,0,0,0,0,0,5.94,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.94,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# We transposed the table so that we can have the item id as the columns' name.
df_revenue = df_revenue.T
df_revenue

id,FOODS_1_001_CA_1,FOODS_1_001_CA_2,FOODS_1_001_CA_3,FOODS_1_001_CA_4,FOODS_1_001_TX_1,FOODS_1_001_TX_2,FOODS_1_001_TX_3,FOODS_1_001_WI_1,FOODS_1_001_WI_2,FOODS_1_001_WI_3,FOODS_1_002_CA_1,FOODS_1_002_CA_2,FOODS_1_002_CA_3,FOODS_1_002_CA_4,FOODS_1_002_TX_1,FOODS_1_002_TX_2,FOODS_1_002_TX_3,FOODS_1_002_WI_1,FOODS_1_002_WI_2,FOODS_1_002_WI_3,FOODS_1_003_CA_1,FOODS_1_003_CA_2,FOODS_1_003_CA_3,FOODS_1_003_CA_4,FOODS_1_003_TX_1,FOODS_1_003_TX_2,FOODS_1_003_TX_3,FOODS_1_003_WI_1,FOODS_1_003_WI_2,FOODS_1_003_WI_3,FOODS_1_004_CA_1,FOODS_1_004_CA_2,FOODS_1_004_CA_3,FOODS_1_004_CA_4,FOODS_1_004_TX_1,FOODS_1_004_TX_2,FOODS_1_004_TX_3,FOODS_1_004_WI_1,FOODS_1_004_WI_2,FOODS_1_004_WI_3,...,HOUSEHOLD_2_513_CA_1,HOUSEHOLD_2_513_CA_2,HOUSEHOLD_2_513_CA_3,HOUSEHOLD_2_513_CA_4,HOUSEHOLD_2_513_TX_1,HOUSEHOLD_2_513_TX_2,HOUSEHOLD_2_513_TX_3,HOUSEHOLD_2_513_WI_1,HOUSEHOLD_2_513_WI_2,HOUSEHOLD_2_513_WI_3,HOUSEHOLD_2_514_CA_1,HOUSEHOLD_2_514_CA_2,HOUSEHOLD_2_514_CA_3,HOUSEHOLD_2_514_CA_4,HOUSEHOLD_2_514_TX_1,HOUSEHOLD_2_514_TX_2,HOUSEHOLD_2_514_TX_3,HOUSEHOLD_2_514_WI_1,HOUSEHOLD_2_514_WI_2,HOUSEHOLD_2_514_WI_3,HOUSEHOLD_2_515_CA_1,HOUSEHOLD_2_515_CA_2,HOUSEHOLD_2_515_CA_3,HOUSEHOLD_2_515_CA_4,HOUSEHOLD_2_515_TX_1,HOUSEHOLD_2_515_TX_2,HOUSEHOLD_2_515_TX_3,HOUSEHOLD_2_515_WI_1,HOUSEHOLD_2_515_WI_2,HOUSEHOLD_2_515_WI_3,HOUSEHOLD_2_516_CA_1,HOUSEHOLD_2_516_CA_2,HOUSEHOLD_2_516_CA_3,HOUSEHOLD_2_516_CA_4,HOUSEHOLD_2_516_TX_1,HOUSEHOLD_2_516_TX_2,HOUSEHOLD_2_516_TX_3,HOUSEHOLD_2_516_WI_1,HOUSEHOLD_2_516_WI_2,HOUSEHOLD_2_516_WI_3
Day,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
d_1,6,4,2,0,0,0,NaN,0,0,0,0,15.76,7.88,0,NaN,0,0,0,8.28,0,0,23.04,2.88,0,0,0,2.88,11.52,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.94,0,0,0,NaN,18.97,0,18.97,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN,0,11.88,0,NaN,0
d_10,0,2,2,0,2,4,0,2,2,6,0,0,0,7.88,0,0,0,7.88,0,15.76,0,2.88,2.88,0,0,0,0,5.76,2.88,2.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,18.97,0,0,0,0,0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.88,0,0,NaN,NaN,0,0,0,NaN,0
d_100,0,10,2,0,2,2,0,2,0,0,7.88,0,7.88,0,0,0,0,0,0,7.88,0,11.52,0,0,0,0,0,11.52,0,2.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,5.94,0,0,0,0,0,0
d_1000,2.24,0,0,0,0,2.24,0,0,0,0,0,8.88,0,8.88,0,0,0,8.88,0,0,0,0,5.76,2.88,5.76,0,0,0,0,5.76,7.12,5.34,0,1.78,7.12,19.58,0,7.12,12.46,8.9,...,0,0,13.9,0,0,5.56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,5.94,5.94,0,0,0,0,0
d_1001,0,13.44,2.24,0,2.24,0,0,2.24,6.72,0,0,0,0,8.88,0,0,0,0,0,0,0,0,0,0,0,0,2.88,0,0,0,16.02,5.34,0,5.34,12.46,23.14,7.12,16.02,16.02,16.02,...,2.78,0,13.9,0,5.56,0,0,2.78,0,5.56,0,35.88,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5.94,0,0,5.94,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d_995,0,4.48,4.48,0,0,0,4.48,0,0,8.96,0,8.88,0,26.64,0,8.88,0,8.88,0,0,0,2.88,0,0,0,0,0,17.28,0,0,17.8,17.8,0,1.78,8.9,8.9,7.12,12.46,19.58,16.02,...,0,5.56,55.6,0,0,2.78,0,0,0,2.78,0,0,0,0,17.94,0,0,17.94,17.94,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,5.94,11.88,0,0,0,0,0
d_996,0,2.24,0,0,0,0,0,2.24,0,0,0,8.88,0,0,0,17.76,0,8.88,0,0,5.76,2.88,2.88,5.76,0,0,2.88,2.88,0,2.88,5.34,7.12,0,1.78,12.46,14.24,3.56,7.12,16.02,17.8,...,0,0,22.24,0,2.78,5.56,5.56,0,0,2.78,0,0,0,0,0,17.94,0,0,0,17.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,5.94,0,0,0,0,5.94
d_997,0,2.24,0,0,2.24,0,0,0,2.24,0,0,0,0,0,0,8.88,0,26.64,8.88,0,2.88,17.28,0,5.76,0,0,0,14.4,0,0,12.46,7.12,0,3.56,16.02,17.8,0,7.12,17.8,14.24,...,0,5.56,5.

In [0]:
# Renamed all the columns to only leave the category each item of each column belongs to
columns_names = list(df_revenue.columns)
for i in range(len(columns_names)):
  columns_names[i] = columns_names[i].split('_')[0]

df_revenue.columns = columns_names

df_revenue

,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,...,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD
Day,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
d_1,6,4,2,0,0,0,NaN,0,0,0,0,15.76,7.88,0,NaN,0,0,0,8.28,0,0,23.04,2.88,0,0,0,2.88,11.52,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.94,0,0,0,NaN,18.97,0,18.97,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN,0,11.88,0,NaN,0
d_10,0,2,2,0,2,4,0,2,2,6,0,0,0,7.88,0,0,0,7.88,0,15.76,0,2.88,2.88,0,0,0,0,5.76,2.88,2.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,18.97,0,0,0,0,0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.88,0,0,NaN,NaN,0,0,0,NaN,0
d_100,0,10,2,0,2,2,0,2,0,0,7.88,0,7.88,0,0,0,0,0,0,7.88,0,11.52,0,0,0,0,0,11.52,0,2.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,5.94,0,0,0,0,0,0
d_1000,2.24,0,0,0,0,2.24,0,0,0,0,0,8.88,0,8.88,0,0,0,8.88,0,0,0,0,5.76,2.88,5.76,0,0,0,0,5.76,7.12,5.34,0,1.78,7.12,19.58,0,7.12,12.46,8.9,...,0,0,13.9,0,0,5.56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,5.94,5.94,0,0,0,0,0
d_1001,0,13.44,2.24,0,2.24,0,0,2.24,6.72,0,0,0,0,8.88,0,0,0,0,0,0,0,0,0,0,0,0,2.88,0,0,0,16.02,5.34,0,5.34,12.46,23.14,7.12,16.02,16.02,16.02,...,2.78,0,13.9,0,5.56,0,0,2.78,0,5.56,0,35.88,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5.94,0,0,5.94,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d_995,0,4.48,4.48,0,0,0,4.48,0,0,8.96,0,8.88,0,26.64,0,8.88,0,8.88,0,0,0,2.88,0,0,0,0,0,17.28,0,0,17.8,17.8,0,1.78,8.9,8.9,7.12,12.46,19.58,16.02,...,0,5.56,55.6,0,0,2.78,0,0,0,2.78,0,0,0,0,17.94,0,0,17.94,17.94,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,5.94,11.88,0,0,0,0,0
d_996,0,2.24,0,0,0,0,0,2.24,0,0,0,8.88,0,0,0,17.76,0,8.88,0,0,5.76,2.88,2.88,5.76,0,0,2.88,2.88,0,2.88,5.34,7.12,0,1.78,12.46,14.24,3.56,7.12,16.02,17.8,...,0,0,22.24,0,2.78,5.56,5.56,0,0,2.78,0,0,0,0,0,17.94,0,0,0,17.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,5.94,0,0,0,0,5.94
d_997,0,2.24,0,0,2.24,0,0,0,2.24,0,0,0,0,0,0,8.88,0,26.64,8.88,0,2.88,17.28,0,5.76,0,0,0,14.4,0,0,12.46,7.12,0,3.56,16.02,17.8,0,7.12,17.8,14.24,...,0,5.56,5.56,0,2.78,2.78,2.78,2.78,0,0,0,0,17.94,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,5.94,0,0,0,0


In [0]:
# Gouped the table by column name this we we will get the total revenue that each category had on each day
df_revenue = df_revenue.fillna(0) 
df_revenue = df_revenue.groupby(df_revenue.columns, axis=1).sum()
df_revenue

,FOODS,HOBBIES,HOUSEHOLD
Day,,,
d_1,48049.50,9795.95,23805.16
d_10,41641.93,6735.10,16787.67
d_100,37787.27,8630.78,19437.48
d_1000,45809.28,11253.21,25365.20
d_1001,52789.78,12192.61,29198.34
...,...,...,...
d_995,65925.30,14841.52,38122.69
d_996,70331.31,14316.21,39113.09
d_997,52471.07,10991.86,27990.92


In [0]:
# We reset the index so that we could have a column named day that woul match the day column of the calenadr table
df_revenue = df_revenue.reset_index()
df_revenue

,Day,FOODS,HOBBIES,HOUSEHOLD
0,d_1,48049.50,9795.95,23805.16
1,d_10,41641.93,6735.10,16787.67
2,d_100,37787.27,8630.78,19437.48
3,d_1000,45809.28,11253.21,25365.20
4,d_1001,52789.78,12192.61,29198.34
...,...,...,...,...
1908,d_995,65925.30,14841.52,38122.69
1909,d_996,70331.31,14316.21,39113.09
1910,d_997,52471.07,10991.86,27990.92
1911,d_998,46502.22,10256.51,25205.53


In [0]:
# Calculated the total day revenue for all the items
df_revenue["Revenue"] = df_revenue["FOODS"] + df_revenue["HOBBIES"] + df_revenue["HOUSEHOLD"]
df_revenue

,Day,FOODS,HOBBIES,HOUSEHOLD,Revenue
0,d_1,48049.50,9795.95,23805.16,81650.61
1,d_10,41641.93,6735.10,16787.67,65164.70
2,d_100,37787.27,8630.78,19437.48,65855.53
3,d_1000,45809.28,11253.21,25365.20,82427.69
4,d_1001,52789.78,12192.61,29198.34,94180.73
...,...,...,...,...,...
1908,d_995,65925.30,14841.52,38122.69,118889.51
1909,d_996,70331.31,14316.21,39113.09,123760.61
1910,d_997,52471.07,10991.86,27990.92,91453.85
1911,d_998,46502.22,10256.51,25205.53,81964.26


In [0]:
# Here we only use the events and the snap columns, drop all the other columns and fill in all the NA values with an integer.
# 0 for any row that is not a holyday and 1 for the ones that are holiday.
df_cal.drop(columns={'date','wm_yr_wk','weekday','wday','month','year','event_name_1','event_name_2'}, inplace=True)
df_cal['event_type_2'] = df_cal['event_type_2'].fillna(-999)
df_cal.loc[(df_cal['event_type_2'] == -999),'event_type_2'] = 0
df_cal['event_type_1'] = df_cal['event_type_1'].fillna(-999)
df_cal.loc[(df_cal['event_type_1'] == -999),'event_type_1'] = 0
df_cal.loc[(df_cal['event_type_1'] != 0),'event_type_1'] = 1
df_cal.loc[(df_cal['event_type_2'] != 0),'event_type_2'] = 1


In [0]:
# Here we created a new column named holiday and added the two event_type columns and if any row in the new column had a value greater than 1
# we just replaced that with 1 denoting that it is a holiday
df_cal['holiday'] = df_cal['event_type_1'] + df_cal['event_type_1']
df_cal.loc[(df_cal['holiday'] >= 1),'holiday'] = 1
df_cal.drop(columns={'event_type_1','event_type_2'}, inplace=True)

In [0]:
# Here we did the same thing as with the event_type, created a new column and aggregated the values of the snaps columns and set the value to 1 in case that
# it had a value greater than 1
df_cal['snap'] = df_cal['snap_CA'] + df_cal['snap_TX'] + df_cal['snap_WI']
df_cal.loc[(df_cal['snap'] >= 1),'snap'] = 1
df_cal.drop(columns={'snap_CA','snap_TX','snap_WI'}, inplace=True)

In [0]:
# Renamed the d column to Day so that it matches with the df_revenue table
df_cal.rename(columns={"d":"Day"},inplace=True)
df_cal

,Day,holiday,snap
0,d_1,0,0
1,d_2,0,0
2,d_3,0,0
3,d_4,0,1
4,d_5,0,1
...,...,...,...
1964,d_1965,0,1
1965,d_1966,0,0
1966,d_1967,0,0
1967,d_1968,0,0


In [0]:
# We merged the revenue and the cal tables.
df_final = pd.merge(df_revenue, df_cal, on='Day', how='inner')
df_final

,Day,FOODS,HOBBIES,HOUSEHOLD,Revenue,holiday,snap
0,d_1,48049.50,9795.95,23805.16,81650.61,0,0
1,d_10,41641.93,6735.10,16787.67,65164.70,0,1
2,d_100,37787.27,8630.78,19437.48,65855.53,1,1
3,d_1000,45809.28,11253.21,25365.20,82427.69,0,0
4,d_1001,52789.78,12192.61,29198.34,94180.73,0,0
...,...,...,...,...,...,...,...
1908,d_995,65925.30,14841.52,38122.69,118889.51,0,0
1909,d_996,70331.31,14316.21,39113.09,123760.61,0,0
1910,d_997,52471.07,10991.86,27990.92,91453.85,0,0
1911,d_998,46502.22,10256.51,25205.53,81964.26,0,0


In [0]:
# Copy the new_sales_amount table into a new variable
total_sales = new_sales_amount
total_sales

,d_1,d_10,d_100,d_1000,d_1001,d_1002,d_1003,d_1004,d_1005,d_1006,d_1007,d_1008,d_1009,d_101,d_1010,d_1011,d_1012,d_1013,d_1014,d_1015,d_1016,d_1017,d_1018,d_1019,d_102,d_1020,d_1021,d_1022,d_1023,d_1024,d_1025,d_1026,d_1027,d_1028,d_1029,d_103,d_1030,d_1031,d_1032,d_1033,...,d_963,d_964,d_965,d_966,d_967,d_968,d_969,d_97,d_970,d_971,d_972,d_973,d_974,d_975,d_976,d_977,d_978,d_979,d_98,d_980,d_981,d_982,d_983,d_984,d_985,d_986,d_987,d_988,d_989,d_99,d_990,d_991,d_992,d_993,d_994,d_995,d_996,d_997,d_998,d_999
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1,3,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,1,1,0,0,3,1,1,0,0,0,0,0,0,0,9,1,0,0,0,0,0,2,0,4,2,0,0,0,1,0,2,0,0,0,1,0
FOODS_1_001_CA_2,2,1,5,0,6,1,4,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,3,4,1,5,0,0,0,2,0,0,5,2,2,0,0,6,0,1,0,1,0,1,0,2,0,3,1,2,0,3,2,1,1,1,0
FOODS_1_001_CA_3,1,1,1,0,1,3,1,0,1,0,1,3,2,5,3,0,0,0,0,0,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0,4,0,0,0,0,...,0,0,2,0,0,0,0,4,2,0,0,0,0,1,0,0,0,0,0,0,6,4,1,0,0,1,0,0,0,0,0,0,1,0,7,2,0,0,0,0
FOODS_1_001_CA_4,0,0,0,0,0,0,0,0,0,0,0,3,0,2,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,0,0,0,1,0,1,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,1,1,0,5,0,0,0,0,0,0,0,0,0,1
FOODS_1_001_TX_1,0,1,1,0,1,1,4,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,1,0,0,1,0,1,1,0,0,0,1,1,1,3,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_516_TX_2,0,0,0,0,0,2,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1
HOUSEHOLD_2_516_TX_3,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
HOUSEHOLD_2_516_WI_1,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,...,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Saved the transpose of it.
total_sales = total_sales.T
total_sales

id,FOODS_1_001_CA_1,FOODS_1_001_CA_2,FOODS_1_001_CA_3,FOODS_1_001_CA_4,FOODS_1_001_TX_1,FOODS_1_001_TX_2,FOODS_1_001_TX_3,FOODS_1_001_WI_1,FOODS_1_001_WI_2,FOODS_1_001_WI_3,FOODS_1_002_CA_1,FOODS_1_002_CA_2,FOODS_1_002_CA_3,FOODS_1_002_CA_4,FOODS_1_002_TX_1,FOODS_1_002_TX_2,FOODS_1_002_TX_3,FOODS_1_002_WI_1,FOODS_1_002_WI_2,FOODS_1_002_WI_3,FOODS_1_003_CA_1,FOODS_1_003_CA_2,FOODS_1_003_CA_3,FOODS_1_003_CA_4,FOODS_1_003_TX_1,FOODS_1_003_TX_2,FOODS_1_003_TX_3,FOODS_1_003_WI_1,FOODS_1_003_WI_2,FOODS_1_003_WI_3,FOODS_1_004_CA_1,FOODS_1_004_CA_2,FOODS_1_004_CA_3,FOODS_1_004_CA_4,FOODS_1_004_TX_1,FOODS_1_004_TX_2,FOODS_1_004_TX_3,FOODS_1_004_WI_1,FOODS_1_004_WI_2,FOODS_1_004_WI_3,...,HOUSEHOLD_2_513_CA_1,HOUSEHOLD_2_513_CA_2,HOUSEHOLD_2_513_CA_3,HOUSEHOLD_2_513_CA_4,HOUSEHOLD_2_513_TX_1,HOUSEHOLD_2_513_TX_2,HOUSEHOLD_2_513_TX_3,HOUSEHOLD_2_513_WI_1,HOUSEHOLD_2_513_WI_2,HOUSEHOLD_2_513_WI_3,HOUSEHOLD_2_514_CA_1,HOUSEHOLD_2_514_CA_2,HOUSEHOLD_2_514_CA_3,HOUSEHOLD_2_514_CA_4,HOUSEHOLD_2_514_TX_1,HOUSEHOLD_2_514_TX_2,HOUSEHOLD_2_514_TX_3,HOUSEHOLD_2_514_WI_1,HOUSEHOLD_2_514_WI_2,HOUSEHOLD_2_514_WI_3,HOUSEHOLD_2_515_CA_1,HOUSEHOLD_2_515_CA_2,HOUSEHOLD_2_515_CA_3,HOUSEHOLD_2_515_CA_4,HOUSEHOLD_2_515_TX_1,HOUSEHOLD_2_515_TX_2,HOUSEHOLD_2_515_TX_3,HOUSEHOLD_2_515_WI_1,HOUSEHOLD_2_515_WI_2,HOUSEHOLD_2_515_WI_3,HOUSEHOLD_2_516_CA_1,HOUSEHOLD_2_516_CA_2,HOUSEHOLD_2_516_CA_3,HOUSEHOLD_2_516_CA_4,HOUSEHOLD_2_516_TX_1,HOUSEHOLD_2_516_TX_2,HOUSEHOLD_2_516_TX_3,HOUSEHOLD_2_516_WI_1,HOUSEHOLD_2_516_WI_2,HOUSEHOLD_2_516_WI_3
d_1,3,2,1,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,1,0,0,8,1,0,0,0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
d_10,0,1,1,0,1,2,0,1,1,3,0,0,0,1,0,0,0,1,0,2,0,1,1,0,0,0,0,2,1,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0
d_100,0,5,1,0,1,1,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,4,0,0,0,0,0,4,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
d_1000,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,2,1,2,0,0,0,0,2,4,3,0,1,4,11,0,4,7,5,...,0,0,5,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
d_1001,0,6,1,0,1,0,0,1,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,9,3,0,3,7,13,4,9,9,9,...,1,0,5,0,2,0,0,1,0,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d_995,0,2,2,0,0,0,2,0,0,4,0,1,0,3,0,1,0,1,0,0,0,1,0,0,0,0,0,6,0,0,10,10,0,1,5,5,4,7,11,9,...,0,2,20,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0
d_996,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,2,0,1,0,0,2,1,1,2,0,0,1,1,0,1,3,4,0,1,7,8,2,4,9,10,...,0,0,8,0,1,2,2,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
d_997,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,3,1,0,1,6,0,2,0,0,0,5,0,0,7,4,0,2,9,10,0,4,10,8,...,0,2,2,0,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
d_998,1,1,0,0,0,0,0,2,0,0,0,1,1,0,0,0,0,1,0,0,2,8,0,2,1,0,0,0,1,0,6,5,0,3,5,14,0,1,2,14,...,0,0,6,0,2,0,1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [0]:
# Renamed all the columns to only leave the category each item of each column belongs to
columns_names = list(total_sales.columns)
for i in range(len(columns_names)):
  columns_names[i] = columns_names[i].split('_')[0]

total_sales.columns = columns_names

total_sales

,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,...,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD,HOUSEHOLD
d_1,3,2,1,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,1,0,0,8,1,0,0,0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
d_10,0,1,1,0,1,2,0,1,1,3,0,0,0,1,0,0,0,1,0,2,0,1,1,0,0,0,0,2,1,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0
d_100,0,5,1,0,1,1,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,4,0,0,0,0,0,4,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
d_1000,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,2,1,2,0,0,0,0,2,4,3,0,1,4,11,0,4,7,5,...,0,0,5,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
d_1001,0,6,1,0,1,0,0,1,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,9,3,0,3,7,13,4,9,9,9,...,1,0,5,0,2,0,0,1,0,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d_995,0,2,2,0,0,0,2,0,0,4,0,1,0,3,0,1,0,1,0,0,0,1,0,0,0,0,0,6,0,0,10,10,0,1,5,5,4,7,11,9,...,0,2,20,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0
d_996,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,2,0,1,0,0,2,1,1,2,0,0,1,1,0,1,3,4,0,1,7,8,2,4,9,10,...,0,0,8,0,1,2,2,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
d_997,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,3,1,0,1,6,0,2,0,0,0,5,0,0,7,4,0,2,9,10,0,4,10,8,...,0,2,2,0,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
d_998,1,1,0,0,0,0,0,2,0,0,0,1,1,0,0,0,0,1,0,0,2,8,0,2,1,0,0,0,1,0,6,5,0,3,5,14,0,1,2,14,...,0,0,6,0,2,0,1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [0]:
# Once again grouped by column name and aggregated the total amount of sales each category had on each day.
total_sales = total_sales.groupby(total_sales.columns, axis=1).sum()
total_sales

,FOODS,HOBBIES,HOUSEHOLD
d_1,23178.0,3764.0,5689.0
d_10,18901.0,2615.0,4056.0
d_100,16383.0,2880.0,4425.0
d_1000,19276.0,3235.0,6749.0
d_1001,22628.0,3476.0,7773.0
...,...,...,...
d_995,27938.0,4242.0,9863.0
d_996,29137.0,4211.0,10004.0
d_997,21483.0,3059.0,7317.0
d_998,19129.0,3236.0,6592.0


In [0]:
# Reset the index
total_sales = total_sales.reset_index()
total_sales

,index,FOODS,HOBBIES,HOUSEHOLD
0,d_1,23178.0,3764.0,5689.0
1,d_10,18901.0,2615.0,4056.0
2,d_100,16383.0,2880.0,4425.0
3,d_1000,19276.0,3235.0,6749.0
4,d_1001,22628.0,3476.0,7773.0
...,...,...,...,...
1908,d_995,27938.0,4242.0,9863.0
1909,d_996,29137.0,4211.0,10004.0
1910,d_997,21483.0,3059.0,7317.0
1911,d_998,19129.0,3236.0,6592.0


In [0]:
# Renamed the columns 
total_sales.columns = ["Day","Food_Sales","Hobbies_Sales","Household_Sales"]
total_sales

,Day,Food_Sales,Hobbies_Sales,Household_Sales
0,d_1,23178.0,3764.0,5689.0
1,d_10,18901.0,2615.0,4056.0
2,d_100,16383.0,2880.0,4425.0
3,d_1000,19276.0,3235.0,6749.0
4,d_1001,22628.0,3476.0,7773.0
...,...,...,...,...
1908,d_995,27938.0,4242.0,9863.0
1909,d_996,29137.0,4211.0,10004.0
1910,d_997,21483.0,3059.0,7317.0
1911,d_998,19129.0,3236.0,6592.0


In [0]:
# Merged the df_final table with total_sales table
df_final = pd.merge(df_final, total_sales, on='Day', how='inner')
df_final

,Day,FOODS,HOBBIES,HOUSEHOLD,Revenue,holiday,snap,Food_Sales,Hobbies_Sales,Household_Sales
0,d_1,48049.50,9795.95,23805.16,81650.61,0,0,23178.0,3764.0,5689.0
1,d_10,41641.93,6735.10,16787.67,65164.70,0,1,18901.0,2615.0,4056.0
2,d_100,37787.27,8630.78,19437.48,65855.53,1,1,16383.0,2880.0,4425.0
3,d_1000,45809.28,11253.21,25365.20,82427.69,0,0,19276.0,3235.0,6749.0
4,d_1001,52789.78,12192.61,29198.34,94180.73,0,0,22628.0,3476.0,7773.0
...,...,...,...,...,...,...,...,...,...,...
1908,d_995,65925.30,14841.52,38122.69,118889.51,0,0,27938.0,4242.0,9863.0
1909,d_996,70331.31,14316.21,39113.09,123760.61,0,0,29137.0,4211.0,10004.0
1910,d_997,52471.07,10991.86,27990.92,91453.85,0,0,21483.0,3059.0,7317.0
1911,d_998,46502.22,10256.51,25205.53,81964.26,0,0,19129.0,3236.0,6592.0


In [0]:
# Saved the final result into a csv file
df_final.to_csv("bayesian_dataset.csv")